<div align="right">
Massimo Nocentini<br>
<br>May 29, 2016: Linearization
<br>May 28, 2016: Binary Forests, Sigmoids
<br>May 27, 2016: Binary and Ternary Trees, Dick paths
<br>May 26, 2016: structure generation
</div>
<br>
<div align="center">
<b>Abstract</b><br>
This document collect some examples about *recursively-defined structures*.
</div>

In [383]:
%matplotlib inline
%run ../python-libs/bits.py
%run ../python-libs/timing.py
%run ../python-libs/graycodes.py
%run ../python-libs/symbols.py

In [455]:
from collections import namedtuple

Anchor = namedtuple('Anchor', ['symbol', 'stars'])
Star = namedtuple('Star', ['row', 'col', 'offsets', 'link', 'symbol'])

# numbering and coordinates are inverses the one of the other                  
def coordinates(group):
    
    def do(anchor):
        c, r = divmod(anchor, group)
        return r, c # simply flip them

    return do

def numbering(group):
    
    def do(coordinate):
        r, c = coordinate
        return r + group*c

    return do

#_______________________________________________________________

def recursive_structures(shapes_spec, start_cell, locating):
    "Returns a generator of structures according `shapes` recursion schemas, starting at anchor `init`."
    
    from collections import defaultdict, deque
    
    shapes_descr, initial_shape = shapes_spec
    coordinates, numbering = locating
            
    def gen(generation):

        yield generation

        next_generation = []
        for anchors, stars, shapes, (anchors_table, stars_table) in generation:
            while stars: # `stars` is an integer to be used as bitmask: 1 in position k 
                         # means that a ★ is present at cell numbered k
                
                anchor, stars = low_bit_and_clear(stars)
                r, c = coordinates(anchor)
                
                # get the corresponding `shape` object
                s = shapes.popleft()
                anchor_tuple = s(r, c)
            
                current_anchors_table, current_stars_table = {}, {}
                current_anchors_table.update(anchors_table)
            
                del stars_table[r, c]
                current_stars_table.update(stars_table)
                
                # remember the anchor symbol
                current_anchors_table[r, c] = anchor_tuple.symbol
                
                # preparing for looping on stars
                augmented_stars = stars
                new_shapes = deque()
                
                for star in anchor_tuple.stars:
                    rs, cs, offsets = star.row, star.col, star.offsets
                    
                    if offsets:
                        r_offset, c_offset = offsets
                        translating_stars = ones_of(augmented_stars)
                        augmented_stars = 0
                        inner_stars_table = {}
                        for ts in translating_stars:
                            ts_r, ts_c = coordinates(ts)
                            symbol = current_stars_table[ts_r, ts_c]
                            translated = ts_r + r_offset, ts_c + c_offset
                            augmented_stars = set_bit(augmented_stars, numbering(translated))
                            inner_stars_table[translated]=symbol
                        current_stars_table = inner_stars_table

                    current_stars_table[rs, cs] = star.symbol
                    new_shapes.append(shapes_descr[star.link])
                    star_cell = numbering((rs,cs),)
                    augmented_stars = set_bit(augmented_stars, star_cell)
                
                struct = (set_bit(anchors, anchor), augmented_stars, new_shapes + shapes, 
                          (current_anchors_table, current_stars_table))
                next_generation.append(struct)

        yield from gen(next_generation)
                
            
    initial_structure = (0b0, 1 << start_cell, 
                         deque([shapes_descr[initial_shape]]), 
                         ({}, {coordinates(start_cell):'★'}))
    
    return gen([initial_structure])

#________________________________________________________________________________________
def make_pretty(dim, joiner=' '):
    
    def pretty(structures,):
        from collections import defaultdict

        rows, cols = dim

        strings = []
        for anchors, stars, shapes, (anchors_table, stars_table) in structures:

            table = defaultdict(lambda: joiner)
            
            for k, v in anchors_table.items():
                table[k] = table[k] + '|' + v if k in table else v

            for k, v in stars_table.items():
                table[k] = table[k] + '|' + v if k in table else v

            s = ''
            for r in range(rows):
                s += joiner.join(table[r, c] for c in range(cols)) + '\n'

            strings.append(s)

        return  ''.join(strings)
    
    return pretty

# Triangulated polygons

In [436]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
★▲★  :nord

★▲   :sw
 ★
 
▲★   :se
★

 ★
★▼   :nw

★
▼★   :ne

★▼★  :south

"""
shapes = {
    'north': lambda r, c: Anchor(symbol='▲', stars=[
                Star(row=r, col=c-1, offsets=None, link='nw', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='ne', symbol='★'),]),
    'sw':   lambda r, c: Anchor(symbol='▲', stars=[
                Star(row=r, col=c-1, offsets=None, link='nw', symbol='★'),
                Star(row=r+1, col=c, offsets=None, link='south', symbol='★'),]),
    'se':   lambda r, c: Anchor(symbol='▲', stars=[
                Star(row=r+1, col=c, offsets=None, link='south', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='ne', symbol='★'),]),
    'nw':   lambda r, c: Anchor(symbol='▼', stars=[
                Star(row=r, col=c-1, offsets=None, link='sw', symbol='★'),
                Star(row=r-1, col=c, offsets=None, link='north', symbol='★'),]),
    'ne':   lambda r, c: Anchor(symbol='▼', stars=[
                Star(row=r-1, col=c, offsets=None, link='north', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='se', symbol='★'),]),
    'south': lambda r, c: Anchor(symbol='▼', stars=[
                Star(row=r, col=c-1, offsets=None, link='sw', symbol='★'),
                Star(row=r, col=c+1, offsets=None, link='se', symbol='★'),]),
}

In [373]:
# skip this, now the above pretty printer should work
def make_pretty_old(dim, joiner=' '):
    
    from collections import defaultdict

    rows, cols = dim
    
    def pretty(structures):
        strings = []
        for anchors, stars, shapes, (anchors_table, stars_table) in structures:
            #print(anchors_table)
            table = defaultdict(lambda: joiner)

            for a in ones_of(anchors):
                ca = coor(a)
                table[ca] = table[ca] + '|' + anchors_table[ca] if ca in table else anchors_table[ca]

            for s in ones_of(stars):
                cs = coor(s)
                table[cs] = table[cs] + '|★' if cs in table else '★'

            s = ''
            for r in range(rows):
                s += joiner.join(table[r, c] for c in range(cols)) + '\n'

            strings.append(s)

        return  ''.join(strings)
    
    return pretty

In [437]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
bin_trees = recursive_structures((shapes, 'north'), rows*12+4, (coor, number))
representations = map(make_pretty((rows, cols),), bin_trees)

First generation contains only the *seed* symbol ★

In [438]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                        ★                        
                                                 
                                                 
                                                 



Next generation:

In [439]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                      ★ ▲ ★                      
                                                 
                                                 
                                                 



Next generation:

In [440]:
print(next(representations))

                                                 
                                                 
                                                 
                      ★                          
                    ★ ▼ ▲ ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                          ★                      
                        ▲ ▼ ★                    
                                                 
                                                 
                                                 



Next generation:

In [441]:
print(next(representations))

                                                 
                                                 
                                                 
                      ★                          
                  ★ ▲ ▼ ▲ ★                      
                    ★                            
                                                 
                                                 
                                                 
                                                 
                                                 
                    ★ ▲ ★                        
                      ▼ ▲ ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                          ★                      


Next generation:

In [442]:
print(next(representations))

                                                 
                                                 
                                                 
                  ★   ★                          
                ★ ▼ ▲ ▼ ▲ ★                      
                    ★                            
                                                 
                                                 
                                                 
                                                 
                                                 
                      ★                          
                    ▲ ▼ ▲ ★                      
                  ★ ▼ ★                          
                                                 
                                                 
                                                 
                                                 
                                                 
                    ★ ▲ ★                        


# Binary trees

In [456]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ●
★ o ★  
"""
shapes = {
    'bintree': lambda r, c: Anchor(symbol='●', stars=[
                Star(row=r+1, col=c-1, offsets=None, link='bintree', symbol='★'),
                Star(row=r+1, col=c+1, offsets=None, link='bintree', symbol='★'),]),
    }

In [457]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
bin_trees = recursive_structures((shapes, 'bintree'), rows*12, (coor, number))
representations = map(make_pretty((rows, cols),), bin_trees)

First generation contains only the *seed* symbol ★

In [458]:
print(next(representations))

                        ★                        
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [459]:
print(next(representations))

                        ●                        
                      ★   ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [460]:
print(next(representations))

                        ●                        
                      ●   ★                      
                    ★   ★                        
                                                 
                                                 
                                                 
                                                 
                                                 
                        ●                        
                          ●                      
                        ★   ★                    
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [461]:
print(next(representations))

                        ●                        
                      ●   ★                      
                    ●   ★                        
                  ★   ★                          
                                                 
                                                 
                                                 
                                                 
                        ●                        
                      ●   ★                      
                        ●                        
                      ★   ★                      
                                                 
                                                 
                                                 
                                                 
                        ●                        
                      ●   ●                      
                        ★   ★                    
                                                 


Next generation:

In [462]:
print(next(representations))

                        ●                        
                      ●   ★                      
                    ●   ★                        
                  ●   ★                          
                ★   ★                            
                                                 
                                                 
                                                 
                        ●                        
                      ●   ★                      
                    ●   ★                        
                      ●                          
                    ★   ★                        
                                                 
                                                 
                                                 
                        ●                        
                      ●   ★                      
                    ●   ●                        
                      ★   ★                      


# Ternary trees

In [409]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ●
★ ★ ★  
"""
threetree_shape = lambda r, c: [((r+1, c-1), (r+1, c), (r+1, c+1))]
shapes = {
    'threetree': lambda r, c: Anchor(symbol='●', stars=[
                Star(row=r+1, col=c-1, offsets=None, link='threetree', symbol='★'),
                Star(row=r+1, col=c, offsets=None, link='threetree', symbol='★'),
                Star(row=r+1, col=c+1, offsets=None, link='threetree', symbol='★'),]),
    }

In [416]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
three_trees = recursive_structures((shapes, 'threetree'), rows*12, (coor, number))
representations = map(make_pretty((rows, cols),), three_trees)

First generation contains only the *seed* symbol ★

In [417]:
print(next(representations))

                        ★                        
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [418]:
print(next(representations))

                        ●                        
                      ★ ★ ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 



Next generation:

In [419]:
print(next(representations))

                        ●                        
                      ● ★ ★                      
                    ★ ★ ★                        
                                                 
                                                 
                                                 
                                                 
                                                 
                        ●                        
                        ● ★                      
                      ★ ★ ★                      
                                                 
                                                 
                                                 
                                                 
                                                 
                        ●                        
                          ●                      
                        ★ ★ ★                    
                                                 


Next generation:

In [420]:
print(next(representations))

                        ●                        
                      ● ★ ★                      
                    ● ★ ★                        
                  ★ ★ ★                          
                                                 
                                                 
                                                 
                                                 
                        ●                        
                      ● ★ ★                      
                      ● ★                        
                    ★ ★ ★                        
                                                 
                                                 
                                                 
                                                 
                        ●                        
                      ● ● ★                      
                      ★ ★ ★                      
                                                 


# Dick paths

In [463]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ★
( o ★  
"""
shapes = {
    'dick': lambda r, c: Anchor(symbol='(', stars=[
                Star(row=r-1, col=c+1, offsets=(0, 2), link='dick', symbol='★'),
                Star(row=r, col=c+2, offsets=None, link='dick', symbol='★'),]),
    }

In [464]:
rows, cols = 8, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
dick_paths = recursive_structures((shapes, 'dick'), rows-1, (coor, number))
representations = map(make_pretty((rows, cols),), dick_paths)

First generation contains only the *seed* symbol ★

In [465]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
★                                                



Next generation:

In [466]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ★                                              
(   ★                                            



Next generation:

In [467]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
                                                 
    ★                                            
  (   ★                                          
(       ★                                        
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
      ★                                          
(   (   ★                                        



Next generation:

In [468]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
      ★                                          
    (   ★                                        
  (       ★                                      
(           ★                                    
                                                 
                                                 
                                                 
                                                 
                                                 
        ★                                        
  (   (   ★                                      
(           ★                                    
                                                 
                                                 
                                                 
                                                 


Next generation:

In [469]:
print(next(representations))

                                                 
                                                 
                                                 
        ★                                        
      (   ★                                      
    (       ★                                    
  (           ★                                  
(               ★                                
                                                 
                                                 
                                                 
                                                 
          ★                                      
    (   (   ★                                    
  (           ★                                  
(               ★                                
                                                 
                                                 
                                                 
                                                 


# Sigmoids

In [471]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

"""
  ★
― o o
    ★
"""
shapes = {
    'sigmoid': lambda r, c: Anchor(symbol='―', stars=[
                Star(row=r-1, col=c+1, offsets=None, link='sigmoid', symbol='★'),
                Star(row=r+1, col=c+2, offsets=None, link='sigmoid', symbol='★'),]),
    }

In [472]:
rows, cols = 9, 25
coor, number = coordinates(rows), numbering(rows)

# starts at the first cell of the 13th column
sigmoids = recursive_structures((shapes, 'sigmoid'), 4, (coor, number))
representations = map(make_pretty((rows, cols),), sigmoids)

First generation contains only the *seed* symbol ★

In [473]:
print(next(representations))

                                                 
                                                 
                                                 
                                                 
★                                                
                                                 
                                                 
                                                 
                                                 



Next generation:

In [474]:
print(next(representations))

                                                 
                                                 
                                                 
  ★                                              
―                                                
    ★                                            
                                                 
                                                 
                                                 



Next generation:

In [475]:
print(next(representations))

                                                 
                                                 
    ★                                            
  ―                                              
―     ★                                          
    ★                                            
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
―     ★                                          
    ―                                            
        ★                                        
                                                 
                                                 



Next generation:

In [476]:
print(next(representations))

                                                 
      ★                                          
    ―                                            
  ―     ★                                        
―     ★                                          
    ★                                            
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 
  ―                                              
―     ★                                          
    ―                                            
        ★                                        
                                                 
                                                 
                                                 
                                                 


Next generation:

In [477]:
print(next(representations))

                                                 
      ★                                          
    ―                                            
  ―     ★                                        
―     ★                                          
    ―                                            
        ★                                        
                                                 
                                                 
        ★                                        
      ―                                          
    ―     ★                                      
  ―     ★                                        
―     ★                                          
                                                 
                                                 
                                                 
                                                 
                                                 
                                                 


# Linearization

In [479]:
# in the following, the symbol ★ has a generation power, 
# namely produces a new structure with the basic shape replacing it.

""" 
● ★ ★  
"""
shapes = {
    'linear': lambda r, c: Anchor(symbol='●', stars=[
                Star(row=r, col=c+1, offsets=(0,2), link='linear', symbol='★'),
                Star(row=r, col=c+2, offsets=None, link='linear', symbol='★'),]),
    }

In [480]:
rows, cols = 3, 25
coor, number = coordinates(rows), numbering(rows)

linears = recursive_structures((shapes, 'linear'), 1, (coor, number))
representations = map(make_pretty((rows, cols),), linears)

First generation contains only the *seed* symbol ★

In [481]:
print(next(representations))

                                                 
★                                                
                                                 



Next generation:

In [482]:
print(next(representations))

                                                 
● ★ ★                                            
                                                 



Next generation:

In [483]:
print(next(representations))

                                                 
● ● ★ ★ ★                                        
                                                 
                                                 
●   ● ★ ★                                        
                                                 



Next generation:

In [484]:
print(next(representations))

                                                 
● ● ● ★ ★ ★ ★                                    
                                                 
                                                 
● ●   ● ★ ★ ★                                    
                                                 
                                                 
● ●     ● ★ ★                                    
                                                 
                                                 
●   ● ● ★ ★ ★                                    
                                                 
                                                 
●   ●   ● ★ ★                                    
                                                 



Next generation:

In [485]:
print(next(representations))

                                                 
● ● ● ● ★ ★ ★ ★ ★                                
                                                 
                                                 
● ● ●   ● ★ ★ ★ ★                                
                                                 
                                                 
● ● ●     ● ★ ★ ★                                
                                                 
                                                 
● ● ●       ● ★ ★                                
                                                 
                                                 
● ●   ● ● ★ ★ ★ ★                                
                                                 
                                                 
● ●   ●   ● ★ ★ ★                                
                                                 
                                                 
● ●   ●     ● ★ ★                                


---
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Recursive Structures tutorial</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="massimo.nocentini@unifi.it" property="cc:attributionName" rel="cc:attributionURL">Massimo Nocentini</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/recursive-structures.ipynb" rel="dct:source">https://github.com/massimo-nocentini/competitive-programming/blob/master/tutorials/recursive-structures.ipynb</a>.